In [5]:
import torch.nn as nn
import torch
import torchvision

torch.manual_seed(42) # так просто
FRAMES = 96 # число кадров в одном видосе
BATCH_SIZE = 1 # размер батча

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class MyResNet18(nn.Module):
    def __init__(self, layers=[3, 4, 6, 3], num_classes = 2):
        super(MyResNet18, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(FRAMES, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(64, layers[0], stride = 1)
        self.layer1 = self._make_layer(128, layers[1], stride = 2)
        self.layer2 = self._make_layer(256, layers[2], stride = 2)
        self.layer3 = self._make_layer(512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)  # решил не добавлять сигмоиды - просто softmax вместо них
    
    def _make_layer(self, planes, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(ResidualBlock(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, num_blocks):
            layers.append(ResidualBlock(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = x.reshape(BATCH_SIZE, FRAMES, x.shape[1], x.shape[2])
        print(f'Размер разбитого видоса: {x.shape}')
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x

In [7]:
video, _, _ = torchvision.io.read_video('video.avi')
print(f'Входной размер видоса: {video.shape}')
FRAMES = video.shape[0]
gray = torchvision.transforms.Grayscale()  # gray и resize тут просто, чтобы видос остался с одним каналом цвета
resize = torchvision.transforms.Resize((256, 256))
video = video.permute((0, 3, 1, 2))
video = resize(gray(video)).squeeze().float()
print(f'На подачу в сетку: {video.shape}')

96


In [8]:
model = MyResNet18()
model(video)

Размер разбитого видоса: torch.Size([1, 96, 256, 256])


tensor([[0.5677, 0.4323]], grad_fn=<SoftmaxBackward0>)